In [1]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Predict House Price/housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [3]:
median_bedrooms = df['total_bedrooms'].median()
df['total_bedrooms'] = df['total_bedrooms'].fillna(median_bedrooms)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20640 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

df['ocean_proximity'] = encoder.fit_transform(df['ocean_proximity'])
df['ocean_proximity'].head()

0    3
1    3
2    3
3    3
4    3
Name: ocean_proximity, dtype: int64

In [5]:
df_labels = df['median_house_value'].values
df_features1 = df.iloc[:,:7]
df_features = pd.concat([df_features1,df['ocean_proximity']],axis =1)
df_features,df_labels

(       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
 0        -122.23     37.88                41.0        880.0           129.0   
 1        -122.22     37.86                21.0       7099.0          1106.0   
 2        -122.24     37.85                52.0       1467.0           190.0   
 3        -122.25     37.85                52.0       1274.0           235.0   
 4        -122.25     37.85                52.0       1627.0           280.0   
 ...          ...       ...                 ...          ...             ...   
 20635    -121.09     39.48                25.0       1665.0           374.0   
 20636    -121.21     39.49                18.0        697.0           150.0   
 20637    -121.22     39.43                17.0       2254.0           485.0   
 20638    -121.32     39.43                18.0       1860.0           409.0   
 20639    -121.24     39.37                16.0       2785.0           616.0   
 
        population  households  ocean_

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_features = scaler.fit_transform(df_features)
df_features

array([[-1.32783522,  1.05254828,  0.98214266, ..., -0.9744286 ,
        -0.97703285,  1.2910888 ],
       [-1.32284391,  1.04318455, -0.60701891, ...,  0.86143887,
         1.66996103,  1.2910888 ],
       [-1.33282653,  1.03850269,  1.85618152, ..., -0.82077735,
        -0.84363692,  1.2910888 ],
       ...,
       [-0.8237132 ,  1.77823747, -0.92485123, ..., -0.3695372 ,
        -0.17404163, -0.11673923],
       [-0.87362627,  1.77823747, -0.84539315, ..., -0.60442933,
        -0.39375258, -0.11673923],
       [-0.83369581,  1.75014627, -1.00430931, ..., -0.03397701,
         0.07967221, -0.11673923]])

In [7]:
ratio = 0.9
m,n = df.shape
X_train = df_features[:round(ratio*m)]
X_test = df_features[round(ratio*m):]
y_train = df_labels[:round(ratio*m)]
y_test = df_labels[round(ratio*m):]
X_train.shape,X_test.shape,y_train.shape, y_test.shape


((18576, 8), (2064, 8), (18576,), (2064,))

In [8]:
#Train Linear SVM
from sklearn.svm import SVR

model = SVR(C = 5,kernel="linear", gamma ='scale')
model

SVR(C=5, kernel='linear')

In [9]:

model.fit(X_train,y_train)

SVR(C=5, kernel='linear')

In [10]:
y_predict = model.predict(X_test)
y_predict

array([185802.11509248, 183081.64712119, 180361.59004592, ...,
       172107.14956248, 171754.22876514, 173377.19899623])

In [11]:
from sklearn.metrics import mean_squared_error
import numpy as np
mse = mean_squared_error(y_predict,y_test)
rmse = np.sqrt(mse)
rmse

89439.41459718897

In [12]:
#Train Linear SVM
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import numpy as np

model = SVR(C = 5, kernel="rbf", gamma ='scale')
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
mse = mean_squared_error(y_predict,y_test)
rmse = np.sqrt(mse)
rmse

92197.60803036072